<a href="https://colab.research.google.com/github/prajwalDU/Pyspark/blob/main/Pyspark_ML_LinearImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. DBFS is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in Python so the default cell type is Python. However, you can use different languages by using the %LANGUAGE syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df =spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

In [ ]:

df.printSchema()

In [ ]:
df.columns

In [ ]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [ ]:
df.show()

In [ ]:
indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r=indexer.fit(df).transform(df)
df_r.show()

In [ ]:
indexer=StringIndexer(inputCols=["smoker","day","time"],outputCols=["smoker_indexed","day_indexed",
                                                                  "time_index"])
df_r=indexer.fit(df_r).transform(df_r)
df_r.show()

In [ ]:
df_r.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed',
                          'time_index'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [ ]:
output.select('Independent Features').show()

In [ ]:
output.show()

In [ ]:
finalized_data=output.select("Independent Features","total_bill")

In [ ]:
finalized_data.show()

In [ ]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [ ]:
regressor.coefficients

In [ ]:
regressor.intercept

In [ ]:
### Predictions
pred_results=regressor.evaluate(test_data)

In [ ]:
## Final comparison
pred_results.predictions.show()

In [ ]:
### PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError